In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
keywordSet = set()
citiesSet = set()
countriesSet = set()
locationsSet = set()
arrayWithMinorKeywords = set()

In [ ]:
def CorrectString(setWithPossibleKeywords, text, considerHashtag):
    word = 'Unknown'
    for each in setWithPossibleKeywords:
        if (re.search('(^|\s|#)' + re.escape(each) + '(?!\S)' , text, re.IGNORECASE)):
            word = each;
            break
    if (considerHashtag and word == 'Unknown'):
        for each in text.split():
            if (each.startswith('#')):
                word = each.lstrip('#')
    return word.lower()

In [ ]:
def fixMissingFields(arrayWithSets, dataframe, nameOfColumnWithParamMissing = 'keyword', nameOfPlaceWithText = 'text', useHashtag = True, writeUnknown = True): 
    for index, row in dataframe.iterrows():
        if (isinstance(row[nameOfColumnWithParamMissing], str)):
            dataframe.loc[index, nameOfColumnWithParamMissing] = row[nameOfColumnWithParamMissing].lower().rstrip().lstrip().replace('%20', ' ')
            continue
        for i in range(len(arrayWithSets)):
            startWritingUnknown = False
            startUsingHashtag = False
            if (i == len(arrayWithSets) - 1):
                startWritingUnknown = writeUnknown
                startUsingHashtag = useHashtag
            wordToPlace = CorrectString(arrayWithSets[i], row[nameOfPlaceWithText], startUsingHashtag)
            if (wordToPlace != 'unknown' or startWritingUnknown):
                dataframe.loc[index, nameOfColumnWithParamMissing] = wordToPlace
                break


In [ ]:
def addToSet(nameOfSet, dataFrameRows):
    for each in dataFrameRows:
        if (each == '' or each.isspace()):
            continue
        nameOfSet.add(each.lower().replace('%20', ' ').lstrip().rstrip())

In [ ]:
#Define the sets and texts
tweets = pd.read_csv('train.csv')
dfWithKeywords = tweets.dropna(subset=['keyword'])
dfWithLocations = tweets.dropna(subset=['location'])
locations = pd.read_csv('worldcities.csv', encoding = 'latin-1')
addToSet(keywordSet, dfWithKeywords['keyword'])
addToSet(locationsSet, dfWithLocations['location'])
addToSet(citiesSet, locations['city'])
addToSet(countriesSet, locations['country'])
arrayWithSets = [locationsSet, citiesSet, countriesSet]
locationsSet.remove('on')
locationsSet.remove('in')
l =[]
for each in locationsSet:
    if (len(each.strip('?')) < 2):
        l.append(each)
for each in l:
    locationsSet.remove(each)
for value in keywordSet:
    for key in keywordSet:
        if (value == key):
            continue
        if value in key:
            arrayWithMinorKeywords.add(value)
            break
for value in arrayWithMinorKeywords:
    keywordSet.remove(value)
    

In [ ]:
#Fix missing fields
fixMissingFields([keywordSet, arrayWithMinorKeywords], tweets, useHashtag=False, writeUnknown = True)
fixMissingFields(arrayWithSets, tweets, 'location', 'text', False)

In [ ]:
tweets.to_csv('withoutEncoding.csv', index = False) #Used to save a csv with that name at the folder where this file is